# Projekt Apache Spark - brudnopis

# Wprowadzenie

Wykorzystując ten notatnik jako szablon zrealizuj projekt Apache Spark zgodnie z przydzielonym zestawem. 

Kilka uwag:

* Nie modyfikuj ani nie usuwaj paragrafów *markdown* w tym notatniku, chyba że wynika to jednoznacznie z instrukcji. 
* Istniejące paragrafy zawierające *kod* uzupełnij w razie potrzeby zgodnie z instrukcjami
    - nie usuwaj ich
    - nie usuwaj zawartych w nich instrukcji oraz kodu
    - nie modyfikuj ich, jeśli instrukcje jawnie tego nie nakazują
* Możesz dodawać nowe paragrafy zarówno zawierające kod jak i komentarze dotyczące tego kodu (markdown)

# Treść projektu

# Zestaw 4 – imdb-persons

## Dwa zbiory danych 

### `datasource1` – informacje na temat najważniejszych osób zaangażowanych w poszczególne filmy (1)

Dane mają format `TSV`, pliki nie posiadają nagłówka.

Pola w pliku:

0. `tconst` – identyfikator filmu
1. `ordering` - numer kolejny osoby w filmie
2. `nconst` - identyfikator osoby
3. `role` - rola osoby w filmie
4. `job` - nazwa zawodu (jeśli dotyczy, w przeciwnym wypadku `\N`)
5. `characters` - nazwa postaci jaką grała osoba (jeśli dotyczy, w przeciwnym wypadku `\N`)

### `datasource4` – informacje na temat osób zaangażowanych w filmach (4)

Dane mają format `TSV`, każdy z plików posiada nagłówek.

Pola w pliku:

0. `nconst` – identyfikator osoby
1. `primaryName` – nazwa (imię i nazwisko) osoby
2. `birthYear` – rok urodzenia
3. `deathYear` – rok śmierci (`\N`, jeśli nie dotyczy)
4. `primaryProfession` – główne profesje osoby
5. `knownForTitles` – identyfikatory filmów, z których ta osoba jest znana

## Misja główna

### Cel przetwarzania 

Dla czterech najbardziej popularnych profesji należy wyznaczyć trzy osoby, które były zaangażowane w największą liczbę filmów zgodnie z tą profesją. 
W obliczeniach nie uwzględniamy filmów, dla których nie ma zdefiniowanej "pełnej obsady". 

Film z pełną obsadą to taki, który posiada: 
- co najmniej jednego aktora (`role in (actor, actress, self)`), 
- co najmniej jednego reżysera (`role = director`) oraz 
- osoby pełniące co najmniej dwie inne dowolne role. 

Ostateczny wynik powinien zawierać następujące atrybuty: 
- `profession` – profesja
- `primaryName` – nazwa osoby
- `movies` – liczba filmów 

Sugerowany schemat wyniku 
```
root
 |-- profession: string (nullable = false)
 |-- primaryName: string (nullable = true)
 |-- movies: long (nullable = false)
```

Uwagi
- Przez profesje rozumiemy wartości występujące jako rozdzielane przecinkami składowe w `primaryProfession` wyłączając z nich wartość `"miscellaneous"`
- Poziom popularności profesji wyznaczany jest wyznaczany jest na podstawie tego ile osób posiada daną profesję na swojej liście profesji w `primaryProfession`
 
## Misje poboczne 

### Misja 1
Przeanalizuj dane dotyczące zmarłych osób wyznacz ile osób żyło określoną liczbę lat. Podaj ilu z nich było aktorami, a ilu reżyserami. 

Wynik ma zawierać następujące kolumny:
- `age` – liczba przeżytych lat
- `persons` – liczba osób, które przeżyły określoną liczbę lat
- `actors` – liczba aktorów (`primaryProfession` zawiera jedną z wartości: `actors`, `actress`). 
- `directors` - liczba reżyserów (`primaryProfession` zawiera wartość `director`).

### Misja 2
Wśród osób, które urodziły się w ubiegłym wieku i które przeżyły ponad 70 lat, wyznacz te trzy, które za brały udział w największej liczbie filmów. Określ w ilu filmach byli oni aktorami oraz ile z tych filmów zostało przez nich wyreżyserowane. 

Wynik ma zawierać następujące kolumny:
- `primaryName` – nazwa (imię i nazwisko) osoby
- `birthYear` – data urodzenia 
- `age` – wiek 
- `filmCount` – liczba filmów w których osoba brała udział
- `filmCountAsActor` – liczba filmów, w których osoba była aktorem/aktorką (role in (`actors`, `actress`, `self`)). 
- `FilmCountAsDirector` - liczba filmów, w których osoba była reżyserem/reżyserką (`role` = `director`).




In [ ]:
Oczekiwany wynik dla misji głównej:
+----------+------------------+------+
|profession|       primaryName|movies|
+----------+------------------+------+
|     actor|Luis Eduardo Motoa|  3559|
|     actor|         Ronit Roy|  2602|
|     actor|       Dilip Joshi|  2385|
|   actress|Luz Stella Luengas|  3636|
|   actress| Rohini Hattangadi|  3240|
|   actress|        Kavita Lad|  3204|
|  producer|     Shobha Kapoor| 11833|
|  producer|       Ekta Kapoor|  8826|
|  producer| Valentin Pimstein|  6081|
|    writer|       Tony Warren|  6153|
|    writer|      Delia Fiallo|  6132|
|    writer|     Sampurn Anand|  5205|
+----------+------------------+------+

Miejsca docelowe dla wyników misji głównych:
* Spark Core - RDD: katalog HDFS /tmp/output1, pliki w formacie SequenceFile serializowane przez Pickle (saveAsPickleFile).
* Spark Core - DataFrame: tabela Delta Lake output2.
* Spark SQL - Pandas API on Spark: plik /tmp/output3.json w lokalnym systemie plików w formacie json (json lines).

# Zestaw 0 – wzorzec

**Uwaga**

- W ramach wzorca nie są spełnione żadne reguły projektu. 
- Brak konsekwencji w wykorzystaniu właściwego API w ramach poszczególnych części
- Zadanie *misji głównej* polega na zliczeniu słówek.  

# Działania wstępne 

Uruchom poniższy paragraf, aby utworzyć obiekty kontekstu Sparka. Jeśli jest taka potrzeba dostosuj te polecenia. Pamiętaj o potrzebnych bibliotekach.

In [7]:
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

W poniższym paragrafie uzupełnij polecenia definiujące poszczególne zmienne. 

Pamiętaj abyś:

* w późniejszym kodzie, dla wszystkich cześci projektu, korzystał z tych zdefiniowanych zmiennych. Wykorzystuj je analogicznie jak parametry
* przed ostateczną rejestracją projektu usunął ich wartości, tak aby nie pozostawiać w notatniku niczego co mogłoby identyfikować Ciebie jako jego autora

In [1]:
# pełna ścieżka do katalogu w zasobniku zawierającego podkatalogi `datasource1` i `datasource4` 
# z danymi źródłowymi
input_dir = "/local_files/input" # BRUDNOPIS

In [2]:
!ls /local_files

input


Nie modyfikuj poniższych paragrafów. Wykonaj je i używaj zdefniowanych poniżej zmiennych jak parametrów Twojego programu.

In [3]:
# NIE ZMIENIAĆ
# ścieżki dla danych źródłowych 
datasource1_dir = input_dir + "/datasource1"
datasource4_dir = input_dir + "/datasource4"

# nazwy i ścieżki dla wyników dla misji głównej 
# część 1 (Spark Core - RDD) 
rdd_result_dir = "/tmp/output1"

# część 2 (Spark SQL - DataFrame)
df_result_table = "output2"

# część 3 (Pandas API on Spark)
ps_result_file = "/tmp/output3.json"

In [4]:
# NIE ZMIENIAĆ
import os
def remove_file(file):
    if os.path.exists(file):
        os.remove(file)

remove_file("metric_functions.py")
remove_file("tools_functions.py")

In [14]:
# NIE ZMIENIAĆ
import requests
r = requests.get("https://jankiewicz.pl/bigdata/metric_functions.py", allow_redirects=True)
open('metric_functions.py', 'wb').write(r.content)
r = requests.get("https://jankiewicz.pl/bigdata/tools_functions.py", allow_redirects=True)
open('tools_functions.py', 'wb').write(r.content)

3322

In [ ]:
# BRUDNOPIS
from metric_functions import *
from tools_functions import *

In [15]:
# NIE ZMIENIAĆ
%run metric_functions.py
%run tools_functions.py

Poniższe paragrafy mają na celu usunąć ewentualne pozostałości poprzednich uruchomień tego lub innych notatników

In [9]:
# NIE ZMIENIAĆ
# usunięcie miejsca docelowego dla część 1 (Spark Core - RDD) 
delete_dir(spark, rdd_result_dir)

Successfully deleted directory: /tmp/output1


In [10]:
# NIE ZMIENIAĆ
# usunięcie miejsca docelowego dla część 2 (Spark SQL - DataFrame) 
drop_table(spark, df_result_table)

The table output2 does not exist.
Path file:/home/jovyan/spark-warehouse/output2 does not exist.


In [16]:
# NIE ZMIENIAĆ
# usunięcie miejsca docelowego dla część 3 (Pandas API on Spark) 
remove_file(ps_result_file)

NameError: name 'remove_file' is not defined

In [19]:
# NIE ZMIENIAĆ
spark

***Uwaga!***

Uruchom poniższy paragraf i sprawdź czy adres, pod którym dostępny *Apache Spark Application UI* jest poprawny wywołując następny testowy paragraf. 

W razie potrzeby określ samodzielnie poprawny adres, pod którym dostępny *Apache Spark Application UI*

In [18]:
# adres URL, pod którym dostępny Apache Spark Application UI (REST API)
# 
spark_ui_address = extract_host_and_port(spark, "http://localhost:4040")
spark_ui_address

'http://localhost:4040'

In [6]:
# testowy paragraf
test_metrics = get_current_metrics(spark_ui_address)
test_metrics

{'numTasks': 0,
 'numActiveTasks': 0,
 'numCompleteTasks': 0,
 'numFailedTasks': 0,
 'numKilledTasks': 0,
 'numCompletedIndices': 0,
 'executorDeserializeTime': 0,
 'executorDeserializeCpuTime': 0,
 'executorRunTime': 0,
 'executorCpuTime': 0,
 'resultSize': 0,
 'jvmGcTime': 0,
 'resultSerializationTime': 0,
 'memoryBytesSpilled': 0,
 'diskBytesSpilled': 0,
 'peakExecutionMemory': 0,
 'inputBytes': 0,
 'inputRecords': 0,
 'outputBytes': 0,
 'outputRecords': 0,
 'shuffleRemoteBlocksFetched': 0,
 'shuffleLocalBlocksFetched': 0,
 'shuffleFetchWaitTime': 0,
 'shuffleRemoteBytesRead': 0,
 'shuffleRemoteBytesReadToDisk': 0,
 'shuffleLocalBytesRead': 0,
 'shuffleReadBytes': 0,
 'shuffleReadRecords': 0,
 'shuffleWriteBytes': 0,
 'shuffleWriteTime': 0,
 'shuffleWriteRecords': 0}

# Część 1 - Spark Core (RDD)

## Misje poboczne

W ponizszych paragrafach wprowadź swoje rozwiązania *misji pobocznych*, o ile **nie** chcesz, aby oceniana była *misja główna*. W przeciwnym przypadku **KONIECZNIE** pozostaw je **puste**.  

## Misja główna 

Poniższy paragraf zapisuje metryki przed uruchomieniem Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [7]:
# NIE ZMIENIAĆ
before_rdd_metrics = get_current_metrics(spark_ui_address)

W poniższych paragrafach wprowadź **rozwiązanie** *misji głównej* oparte na *RDD API*. 

Pamiętaj o wydajności Twojego przetwarzania, *RDD API* tego wymaga. 

Nie wprowadzaj w poniższych paragrafach żadnego kodu, w przypadku wykorzystania *misji pobocznych*.

In [ ]:
# Wczytanie plików tekstowych
actor_data = sc.textFile(datasource4_dir)
movie_crew_data = sc.textFile(datasource1_dir)

In [ ]:

# Filmy z pełną obsadą
movies_data = movie_crew_data.map(lambda line: line.split("\t")).\
    map(lambda fields: (fields[0], fields[2], fields[3])).\
    groupBy(lambda x: x[0])
# Filtr dla filmów
def filter_movies(group):
    movie_id, entries = group
    roles = [entry[2] for entry in entries]
    people = set(entry[1] for entry in entries)  # Unique person_ids
    has_actor = any(role in {"actor", "actress", "self"} for role in roles)
    has_director = "director" in roles
    return len(people) >= 4 and has_actor and has_director

movies_full_cast = movies_data.filter(filter_movies)

# Najpopularniejsze profesje
profession_counts = actor_data.map(lambda line: line.split("\t")[4]).\
    flatMap(lambda professions: professions.split(",")).\
    filter(lambda profession: profession != "miscellaneous").\
    map(lambda profession: (profession, 1)).\
    reduceByKey(lambda x, y: x + y)    
top4_professions = profession_counts.takeOrdered(4)

# Osoby zaangażowane
most_movies_worked_on = None

In [27]:
# Zapis wyniku do pliku pickle
word_counts.saveAsPickleFile(rdd_result_dir)

In [29]:
!ls /tmp/output1
!head /tmp/output1/part-00000

part-00000  part-00004	part-00008  part-00012	part-00016
part-00001  part-00005	part-00009  part-00013	part-00017
part-00002  part-00006	part-00010  part-00014	_SUCCESS
part-00003  part-00007	part-00011  part-00015
SEQ!org.apache.hadoop.io.NullWritable"org.apache.hadoop.io.BytesWritable      [���dbe{pĉp$!�  �      |�� ur [[BK�gg�7  xp   
nm0000158	Tom�K���nm0000159	Teri�K���nm0000166	Helen�K���OJones	1946	\N	actor,director,soundtrack	tt0106977,tt0477348,tt2398231,tt0443272�K���TLocklear	1961	\N	actress,producer,soundtrack	tt0380623,tt0119695,tt0103491,tt0087262�K���nm0000183	Traci�K���PPacino	1940	\N	actor,soundtrack,director	tt0099422,tt0068646,tt0070666,tt0072890�K���nm0000200	Bill�K���QPhillippe	1974	\N	actor,producer,director	tt0202677,tt0280707,tt0139134,tt0375679�K��e.uq ~   ����      ]�(�OPosey	1968	\N	actress,soundtrack,writer	tt0348150,tt0134084,tt0359013,tt0106677�K���PReeves	1964	\N	actor,producer,soundtrack	tt0111257,tt0133093,tt0102685,tt0234215�

Poniższy paragraf zapisuje metryki po uruchomieniu Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [17]:
# NIE ZMIENIAĆ
after_rdd_metrics = get_current_metrics(spark_ui_address)

# Część 2 - Spark SQL (DataFrame)

## Misje poboczne

W ponizszych paragrafach wprowadź swoje rozwiązania *misji pobocznych*, o ile **nie** chcesz, aby oceniana była *misja główna*. W przeciwnym przypadku **KONIECZNIE** pozostaw je **puste**.  

## Misja główna 

Poniższy paragraf zapisuje metryki przed uruchomieniem Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [18]:
# NIE ZMIENIAĆ
before_df_metrics = get_current_metrics(spark_ui_address)

W poniższych paragrafach wprowadź **rozwiązanie** *misji głównej* swojego projektu oparte o *DataFrame API*. 

Pamiętaj o wydajności Twojego przetwarzania, *DataFrame API* nie jest w stanie wszystkiego "naprawić". 

Nie wprowadzaj w poniższych paragrafach żadnego kodu, w przypadku wykorzystania *misji pobocznych*.

In [32]:
from pyspark.sql.functions import col, explode, split, count, desc
# Wczytanie danych
datasource1 = spark.read.option("sep", "\t").csv(datasource1_dir, inferSchema=True)
datasource4 = spark.read.option("sep", "\t").csv(datasource4_dir, header=True, inferSchema=True)
datasource1 = datasource1.toDF("tconst", "ordering", "nconst", "role", "job", "characters")
datasource4 = datasource4.toDF("nconst", "primaryName", "birthYear", "deathYear", "primaryProfession", "knownForTitles")

In [ ]:
from pyspark.sql.functions import collect_set, size, array_contains, row_number, when
from pyspark.sql import Window

normalized_roles = datasource1.withColumn(
    "normalized_role",
    when(col("role").isin("actor", "actress", "self"), "performer").otherwise(col("role"))
)

full_cast = normalized_roles.groupBy("tconst").agg(collect_set("normalized_role").alias("roles")).\
    filter(
        array_contains(col("roles"), "performer") & 
        array_contains(col("roles"), "director") & 
        (size(col("roles")) > 3)
    ).select("tconst")



In [ ]:
# Połączenie z datasource1 dla pełnej obsady
full_cast_roles = full_cast.join(datasource1, "tconst").select("tconst", "nconst", "role")

+---------+---------+---------------+
|   tconst|   nconst|           role|
+---------+---------+---------------+
|tt0000725|nm0226992|          actor|
|tt0000725|nm0366008|          actor|
|tt0000725|nm0000428|         writer|
|tt0000725|nm0005658|cinematographer|
|tt0000725|nm0567363|       director|
|tt0000861|nm0732651|        actress|
|tt0000861|nm0784407|          actor|
|tt0000861|nm0005658|cinematographer|
|tt0000861|nm0163559|          actor|
|tt0000861|nm0000428|       director|
|tt0000861|nm0456804|          actor|
|tt0000861|nm0910400|          actor|
|tt0000861|nm0253652|         writer|
|tt0000861|nm0642722|          actor|
|tt0000861|nm0940488|         writer|
|tt0000862|nm0386036|          actor|
|tt0000862|nm0264569|        actress|
|tt0000862|nm5289829|          actor|
|tt0000862|nm0511080|          actor|
|tt0000862|nm0878467|       director|
+---------+---------+---------------+
only showing top 20 rows



In [ ]:
full_cast_roles_count = full_cast_roles.groupBy("nconst", "role").agg(count("tconst").alias("movies")).\
    select("nconst", col("role").alias("profession"), "movies")

+---------+---------------+------+
|   nconst|     profession|movies|
+---------+---------------+------+
|nm0577476|          actor|    16|
|nm0232704|        actress|     2|
|nm0852794|cinematographer|     2|
|nm0430756|         writer|    66|
|nm0354894|         writer|     7|
|nm0001273|        actress|    57|
|nm0222369|        actress|     2|
|nm0294571|        actress|    12|
|nm0746704|        actress|     4|
|nm0376221|       director|   102|
|nm0415405|        actress|     1|
|nm0319702|cinematographer|    26|
|nm0253296|       director|     7|
|nm0299343|         writer|    52|
|nm0221488|          actor|    69|
|nm0384716|        actress|    21|
|nm0006297|       composer|    37|
|nm0622404|          actor|    16|
|nm0518711|         writer|   193|
|nm0703642|       producer|    94|
+---------+---------------+------+
only showing top 20 rows



In [ ]:
# Przetwarzanie datasource4: Rozdzielanie profesji
actor_data = datasource4.withColumn("profession", explode(split(col("primaryProfession"), ","))).\
    filter(col("profession") != "miscellaneous")

+---------+---------------+---------+---------+--------------------+--------------------+----------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|profession|
+---------+---------------+---------+---------+--------------------+--------------------+----------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0072308,tt00430...|soundtrack|
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0072308,tt00430...|     actor|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0038355,tt01170...|   actress|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0038355,tt01170...|soundtrack|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0057345,tt00544...|   actress|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0057345,tt00544...|soundtrack|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0057345,tt00544...|  

In [ ]:
# Najpopularniejsze profesje
top_professions = actor_data.groupBy("profession").agg(count("nconst").alias("count")).orderBy(desc("count")).limit(4)

+----------+-------+
|profession|  count|
+----------+-------+
|     actor|2259212|
|   actress|1354336|
|  producer| 845163|
|    writer| 641773|
+----------+-------+



In [ ]:
movies_per_person = full_cast_roles_count.join(datasource4.select("nconst", "primaryName"), "nconst").join(top_professions, "profession")
window_spec = Window.partitionBy("profession").orderBy(desc("movies"))
ranked_movies_per_person2 = movies_per_person.withColumn(
    "rank", row_number().over(window_spec)
)
final_result = ranked_movies_per_person2.filter(col("rank") <= 3).\
    select("profession", "primaryName", "movies").orderBy("profession", desc("movies"))


In [ ]:
final_result.show()

+----------+------------------+------+
|profession|       primaryName|movies|
+----------+------------------+------+
|     actor|Luis Eduardo Motoa|  3559|
|     actor|         Ronit Roy|  2602|
|     actor|       Dilip Joshi|  2385|
|   actress|Luz Stella Luengas|  3636|
|   actress| Rohini Hattangadi|  3240|
|   actress|        Kavita Lad|  3204|
|  producer|     Shobha Kapoor| 11833|
|  producer|       Ekta Kapoor|  8826|
|  producer| Valentin Pimstein|  6081|
|    writer|       Tony Warren|  6153|
|    writer|      Delia Fiallo|  6132|
|    writer|     Sampurn Anand|  5205|
+----------+------------------+------+



In [ ]:
# Zapis wyników do tabeli 
final_result.write.mode("overwrite").format("delta").saveAsTable(df_result_table)

Poniższy paragraf zapisuje metryki po uruchomieniu Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [22]:
# NIE ZMIENIAĆ
after_df_metrics = get_current_metrics(spark_ui_address)

# Część 3 - Pandas API on Spark

Ta część to wyzwanie. W szczególności dla osób, które nie programują na co dzień w Pythonie, lub które nie nie korzystały do tej pory z Pandas API.  

Powodzenia!

## Misje poboczne

W ponizszych paragrafach wprowadź swoje rozwiązania *misji pobocznych*, o ile **nie** chcesz, aby oceniana była *misja główna*. W przeciwnym przypadku **KONIECZNIE** pozostaw je **puste**.  

## Misja główna 

Poniższy paragraf zapisuje metryki przed uruchomieniem Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [23]:
#NIE ZMIENIAĆ
before_ps_metrics = get_current_metrics(spark_ui_address)

W poniższych paragrafach wprowadź **rozwiązanie** swojego projektu oparte o *Pandas API on Spark*. 

Pamiętaj o wydajności Twojego przetwarzania, *Pandas API on Spark* nie jest w stanie wszystkiego "naprawić". 

Nie wprowadzaj w poniższych paragrafach żadnego kodu, w przypadku wykorzystania *misji pobocznych*.

In [8]:
import pyspark.pandas as ps

lines_ps = ps.read_csv(datasource4_dir, header=None)

/usr/local/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
/usr/local/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [17]:
words_ps = lines_ps[0].apply(lambda x: x.split(' ') if x is not None else []).explode().reset_index(drop=True)

In [18]:
word_counts = words_ps.value_counts()

In [19]:
word_counts_pandas = word_counts.head(50).to_pandas()

c:\Users\kubag\miniconda3\envs\data_science_env\Lib\site-packages\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas Series is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Py4JJavaError: An error occurred while calling o331.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 14.0 failed 1 times, most recent failure: Lost task 5.0 in stage 14.0 (TID 72) (192.168.56.1 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.ArrowEvalPythonExec.evaluate(ArrowEvalPythonExec.scala:92)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun$ma

In [28]:
word_counts_pandas.to_json(ps_result_file, orient='index')

Poniższy paragraf zapisuje metryki po uruchomieniu Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [30]:
#NIE ZMIENIAĆ
after_ps_metrics = get_current_metrics(spark_ui_address)

# Analiza wyników i wydajności *misji głównych*

## Część 1 - Spark Core (RDD)

In [31]:
# Wczytanie wyników z pliku pickle
word_counts = sc.pickleFile(rdd_result_dir)

# Wyświetlenie 50 pierwszych elementów
result_sample = word_counts.take(50)
for item in result_sample:
    print(item)

('TeriyakiApps\x01~\x01teriyakiapps@gmail.com\x018589934594', 1)
('Koza\x01http://www.xynapse.pl\x01xynapse@xynapse.pl\x018589934595', 1)
('Tools\x01https://vcb30cb43.app-ads-txt.com/app-ads.txt\x01androtools222@gmail.com\x018589934596', 1)
('Muslim', 110)
('FireFlies', 2)
('Studio\x01~\x01manuariza95@gmail.com\x018589934602', 1)
('News', 494)
('IST-Development\x01https://istanbulit.com\x01info@istanbulit.com\x018589934604', 1)
('FAStuidoTI\x01~\x01karimkhalfy@gmail.com\x018589934605', 1)
('Web4Minds,', 1)
('V3', 8)
('Smart', 2437)
('Ltd\x01http://www.v3smarttech.com\x01support@v3smarttech.com\x018589934607', 1)
('Mobil', 143)
('UNDERSCORE:', 1)
('Apps', 6350)
('and', 5289)
('Games\x01~\x01ergamesapps@gmail.com\x018589934609', 1)
('tamapps\x01~\x01zakdermeister@gmail.com\x018589934614', 1)
('S.', 397)
('Connect', 331)
('Team\x01https://mewe.com/join/klwpdevelopersteam\x01designcorpviti@gmail.com\x018589934618', 1)
('for', 2565)
('with', 262)
('NETWORKS', 23)
('PTE', 226)
('Art\x01https

In [32]:
subtract_metrics(after_rdd_metrics, before_rdd_metrics)

{'numTasks': 6,
 'numActiveTasks': 0,
 'numCompleteTasks': 6,
 'numFailedTasks': 0,
 'numKilledTasks': 0,
 'numCompletedIndices': 6,
 'executorDeserializeTime': 763,
 'executorDeserializeCpuTime': 288417800,
 'executorRunTime': 52789,
 'executorCpuTime': 3791290300,
 'resultSize': 12143,
 'jvmGcTime': 1808,
 'resultSerializationTime': 19,
 'memoryBytesSpilled': 0,
 'diskBytesSpilled': 0,
 'peakExecutionMemory': 0,
 'inputBytes': 84276905,
 'inputRecords': 1179547,
 'outputBytes': 90624535,
 'outputRecords': 14566,
 'shuffleRemoteBlocksFetched': 0,
 'shuffleLocalBlocksFetched': 9,
 'shuffleFetchWaitTime': 0,
 'shuffleRemoteBytesRead': 0,
 'shuffleRemoteBytesReadToDisk': 0,
 'shuffleLocalBytesRead': 49730906,
 'shuffleReadBytes': 49730906,
 'shuffleReadRecords': 228,
 'shuffleWriteBytes': 49730906,
 'shuffleWriteTime': 405851800,
 'shuffleWriteRecords': 228}

## Część 2 - Spark SQL (DataFrame)

In [33]:
df = spark.table(df_result_table)

# Wyświetlenie 50 pierwszych rekordów
df.show(50)

+-------------------------+-----+
|                     word|count|
+-------------------------+-----+
|                      The| 9372|
|                   Bidhee|    7|
|                Solutions| 6041|
|                   ArtAce|    2|
|                  PuyTech|    1|
|                   McLeod|  208|
|                      RTV|   13|
|     Softwarehttp://p...|    1|
|紫荊雜誌社https://bau...|    1|
|                  Bacilio|    2|
|     Developerhttps:/...|    1|
|     Softwarehttp://w...|    1|
|                  Backend|   13|
|하이퍼펌프~hyper.cho...|    1|
|                    METRO|   21|
|     ADBANDhttp://www...|    1|
|                      Tcf|    1|
|                      Pug|   12|
|              Techologies|    4|
|     Tourismhttps://t...|    1|
|     Kinsale~gourmet...|    1|
|     Englishhttps://w...|    1|
|                    Darul|   10|
|                       📱|    3|
|                  Panipat|    2|
|     Konyukhovhttp://...|    1|
|                     Bol

In [34]:
subtract_metrics(after_df_metrics, before_df_metrics)

{'numTasks': 12,
 'numActiveTasks': 0,
 'numCompleteTasks': 8,
 'numFailedTasks': 0,
 'numKilledTasks': 0,
 'numCompletedIndices': 8,
 'executorDeserializeTime': 1254,
 'executorDeserializeCpuTime': 446474600,
 'executorRunTime': 54900,
 'executorCpuTime': 22626614900,
 'resultSize': 36185,
 'jvmGcTime': 2428,
 'resultSerializationTime': 110,
 'memoryBytesSpilled': 0,
 'diskBytesSpilled': 0,
 'peakExecutionMemory': 440400752,
 'inputBytes': 84344235,
 'inputRecords': 1179547,
 'outputBytes': 50321941,
 'outputRecords': 1456441,
 'shuffleRemoteBlocksFetched': 0,
 'shuffleLocalBlocksFetched': 16,
 'shuffleFetchWaitTime': 0,
 'shuffleRemoteBytesRead': 0,
 'shuffleRemoteBytesReadToDisk': 0,
 'shuffleLocalBytesRead': 63406957,
 'shuffleReadBytes': 63406957,
 'shuffleReadRecords': 1622698,
 'shuffleWriteBytes': 63406957,
 'shuffleWriteTime': 817119700,
 'shuffleWriteRecords': 1622698}

## Część 3 - Pandas API on Spark

In [15]:
import json

# Odczytaj zawartość pliku JSON
with open(ps_result_file, 'r') as file:
    json_content = json.load(file)

# Wyświetl zawartość
print(json.dumps(json_content, indent=2))

FileNotFoundError: [Errno 2] No such file or directory: 'D:/studia/Semestr 7/Big Data/laby/projekt2/output3.json'

In [36]:
subtract_metrics(after_ps_metrics, before_ps_metrics)

{'numTasks': 33,
 'numActiveTasks': 0,
 'numCompleteTasks': 25,
 'numFailedTasks': 0,
 'numKilledTasks': 0,
 'numCompletedIndices': 25,
 'executorDeserializeTime': 1838,
 'executorDeserializeCpuTime': 440241100,
 'executorRunTime': 166601,
 'executorCpuTime': 55323279000,
 'resultSize': 134363,
 'jvmGcTime': 4753,
 'resultSerializationTime': 123,
 'memoryBytesSpilled': 0,
 'diskBytesSpilled': 0,
 'peakExecutionMemory': 427817888,
 'inputBytes': 385819487,
 'inputRecords': 5409845,
 'outputBytes': 0,
 'outputRecords': 0,
 'shuffleRemoteBlocksFetched': 0,
 'shuffleLocalBlocksFetched': 20,
 'shuffleFetchWaitTime': 0,
 'shuffleRemoteBytesRead': 0,
 'shuffleRemoteBytesReadToDisk': 0,
 'shuffleLocalBytesRead': 61239298,
 'shuffleReadBytes': 61239298,
 'shuffleReadRecords': 1573467,
 'shuffleWriteBytes': 61239298,
 'shuffleWriteTime': 1111152100,
 'shuffleWriteRecords': 1573467}